In [ ]:
from mstrio.connection import Connection
from mstrio.api import transformations
from mstrio.api.browsing import store_search_instance, get_search_results
from mstr_robotics.read_out_prj_obj import read_gen
import json
with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

conn_params =  user_d["conn_params"]
conn = Connection(**conn_params)
project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
base_path="D:/shared_drive/AI_working_folder/Compare_obj/"

In [ ]:
conn.select_project(project_id)
comp_obj_id="98EB31B54122FFB738E6E08A2F29421A%3B55" 
comp_obj_d={"source_obj_d":{"base_url":,"project_id":"","object_id"},"target_obj_d":{}}
url='http://85.214.60.83:8080/MicroStrategyLibrary/api/metadataSearches/results?pattern=4&domain=2&scope=standalone&usedByObject={comp_obj_id}&usedByRecursive=true&visibility=ALL'
resul=conn.post(url).json()
print(resul)
rep_l=get_search_results(
    connection=conn,
    search_id=resul["id"],
    project_id=project_id).json()
dpn_obj_l=[]
for obj in rep_l:
    obj_d={}
    obj_d["id"]=obj["id"]
    obj_d["name"]=obj["name"]
    obj_d["type"]=obj["type"]
    obj_d["subtype"]=obj["subtype"]
    
    dpn_obj_l.append(obj_d.copy())
#dpn_obj_l


In [ ]:
i_read_gen=read_gen()
all_def_l=[]
for o in dpn_obj_l:
    res_d=i_read_gen.get_obj_def(conn=conn, object_id=o["id"], 
                                 obj_type=o["type"],
                                 obj_sub_type=obj_d["subtype"])
    if len(res_d.keys()):
        all_def_l.append(res_d)
#all_def_l

In [ ]:
def remove_keys(obj):
    if isinstance(obj, dict):
        return {
            k: remove_keys(v)
            for k, v in obj.items()
            if k not in keys_to_remove
        }
    elif isinstance(obj, list):
        return [remove_keys(i) for i in obj]
    else:
        return obj

In [ ]:
keys_to_remove = {"dateModified", "dateCreated"}
cleaned_obj_l = []
for obj in all_def_l:
    cleaned_obj = remove_keys(obj)
    cleaned_obj_l.append(cleaned_obj.copy())
cleaned_obj_l

In [ ]:
import os
import json

os.makedirs("cleaned_json", exist_ok=True)

for obj_j in cleaned_obj_l:
    if "information" in obj_j:
        fname = f"{obj_j['information']['subType']}_{obj_j['information']['objectId']}_{obj_j['information']['name']}"
    else:
        fname = f"{obj_j.get('subType', 'unknown')}_{obj_j.get('id', 'unknown')}_{obj_j.get('name', 'unknown')}"
    fname = fname.replace(" ", "")
    file_path = os.path.join(base_path, , f"{fname}.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(obj_j, f, ensure_ascii=False, indent=2)

In [ ]:
import json
from mstrio.connection import Connection
from mstrio.api.browsing import get_search_results
from mstr_robotics.read_out_prj_obj import read_gen

def connect_mstr(conn_params, project_id):
    conn = Connection(**conn_params)
    conn.select_project(project_id)
    return conn


def get_object_definitions(conn, obj_list):
    i_read_gen = read_gen()
    all_defs = []
    for o in obj_list:
        res_d = i_read_gen.get_obj_def(
            conn=conn,
            object_id=o["id"],
            obj_type=o["type"],
            obj_sub_type=o.get("subtype")
        )
        if res_d:
            all_defs.append(res_d)
    return all_defs

def remove_keys(obj, keys_to_remove):
    if isinstance(obj, dict):
        return {k: remove_keys(v, keys_to_remove) for k, v in obj.items() if k not in keys_to_remove}
    elif isinstance(obj, list):
        return [remove_keys(i, keys_to_remove) for i in obj]
    else:
        return obj

def save_objects(obj_list, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    for obj in obj_list:
        fname = f"{obj.get('subType', 'unknown')}_{obj.get('id', 'unknown')}_{obj.get('name', 'unknown')}".replace(" ", "")
        file_path = os.path.join(out_dir, f"{fname}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(obj, f, ensure_ascii=False, indent=2)

def load_objects_from_dir(out_dir):
    obj_def_d = {}
    obj_def_d_l = []
    for fname in os.listdir(out_dir):
        if fname.endswith('.json'):
            with open(os.path.join(out_dir, fname), encoding="utf-8") as f:
                obj_def_d[fname] = json.load(f)
                obj_def_d_l.append(obj_def_d[fname])
    return obj_def_d_l

def compare_object_files(dir1, dir2):
    files1 = set(os.listdir(dir1))
    files2 = set(os.listdir(dir2))
    only_in_1 = files1 - files2
    only_in_2 = files2 - files1
    in_both = files1 & files2

    print("Files only in", dir1, ":", only_in_1)
    print("Files only in", dir2, ":", only_in_2)

    for fname in in_both:
        with open(os.path.join(dir1, fname), encoding="utf-8") as f1, \
                open(os.path.join(dir2, fname), encoding="utf-8") as f2:
            obj1 = json.load(f1)
            obj2 = json.load(f2)
            if obj1 != obj2:
                print(f"Differences found in {fname}")

def mstr_parse_search(conn, search_id):

    rep_l=get_search_results(
        connection=conn,
        search_id=search_id,
        project_id=project_id).json()


    for obj in rep_l:
        obj_d={}
        obj_d["id"]=obj["id"]
        obj_d["name"]=obj["name"]
        obj_d["type"]=obj["type"]
        obj_d["subtype"]=obj["subtype"]
    return obj_d

if __name__ == "__main__":
    # Example input dicts for two systems
    input_dicts = [
        {
            "conn_params": {
                "username": "Administrator",
                "password": "password1",
                "base_url": "http://host1/MicroStrategyLibrary/api"
            },
            "project_id": "PROJECT_ID_1",
            "out_dir": "system1_json"
        },
        {
            "conn_params": {
                "username": "Administrator",
                "password": "password2",
                "base_url": "http://host2/MicroStrategyLibrary/api"
            },
            "object_id": "_ID_2",
            "project_id": "PROJECT_ID_2",
            "out_dir": "system2_json"
        }
    ]

    keys_to_remove = {"dateModified", "dateCreated"}

    for inp in input_dicts:
        conn = connect_mstr(inp["conn_params"], inp["project_id"])
        obj_l = fetch_objects(conn, inp["project_id"])
        all_defs = get_object_definitions(conn, obj_l)
        cleaned_objs = [remove_keys(obj, keys_to_remove) for obj in all_defs]
        save_objects(cleaned_objs, inp["out_dir"])

    # Load and compare
    compare_object_files(input_dicts[0]["out_dir"], input_dicts[1]["out_dir"])


In [ ]:
short_cut_folder_id="986E6D1A427030A710A0E68F74290482"

# Connect Projects
with open('D:\\shared_drive\\Python\\mstr_robotics\\mstr_robotics\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

#set user credentials and open a connection to the i-server
username = user_d["username"]
password =user_d["password"]
base_url= user_d["base_url"]

source_username=username
source_password=password
source_base_url=base_url

target_username=username
target_password=password
target_base_url=base_url

source_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
target_project_id="00F3103E41FF3743A057C3B5313595B7"
source_conn = Connection(base_url=source_base_url,username=source_username,
                  password=source_password,project_id=source_project_id)
source_conn.headers['Content-type'] = "application/json"

target_conn = Connection(base_url=target_base_url,username=target_username,
                  password=target_password,project_id=target_project_id)
target_conn.headers['Content-type'] = "application/json"
comp_obj_id="98EB31B54122FFB738E6E08A2F29421A%3B55" 
comp_obj_d={"source_obj_d":{"source_conn":source_conn,"project_id":source_project_id, "object_id":comp_obj_id},"target_obj_d":{"target_conn":target_conn,"project_id":target_project_id, "object_id":comp_obj_id}}

url='http://85.214.60.83:8080/MicroStrategyLibrary/api/metadataSearches/results?pattern=4&domain=2&scope=standalone&usedByObject={comp_obj_id}&usedByRecursive=true&visibility=ALL'
resul=conn.post(url).json()
obj_l=mstr_parse_search(conn=conn, search_id= resul["id"])
obj_list_def_l=get_object_def(conn=conn, obj_l=obj_l)
save_objects(obj_list_def_l, base_path)


url='http://85.214.60.83:8080/MicroStrategyLibrary/api/metadataSearches/results?pattern=4&domain=2&scope=standalone&usedByObject={comp_obj_id}&usedByRecursive=true&visibility=ALL'

resul=conn.post(url).json()


In [ ]:
import pandas as pd
df=pd.read_csv("D:/shared_drive/Python/mstr_robotics/json_compare/json_diff_analysis.csv")


,id,name,subType,key_path_to_diff,original_value,comparison_value,diff_val,path_to_org_file,path_to_compare_file
0,9F7DEC1147031A48C26DB894CE389C75,simpletest_org,values_changed,id,9F7DEC1147031A48C26DB894CE389C75,560AB68248B52FAA1AF6CB8E529E0993,Value changed from '9F7DEC1147031A48C26DB894CE...,D:\shared_drive\Python\mstr_robotics\json_comp...,D:\shared_drive\Python\mstr_robotics\json_comp...
1,9F7DEC1147031A48C26DB894CE389C75,simpletest_org,values_changed,name,simpletest_org,simpletest_comp,Value changed from 'simpletest_org' to 'simple...,D:\shared_drive\Python\mstr_robotics\json_comp...,D:\shared_drive\Python\mstr_robotics\json_comp...
2,9F7DEC1147031A48C26DB894CE389C75,simpletest_org,values_changed,chapters[0]['pages'][0]['visualizations'][0]['...,Visualization 1,Visualization_compare,Value changed from 'Visualization 1' to 'Visua...,D:\shared_drive\Python\mstr_robotics\json_comp...,D:\shared_drive\Python\mstr_robotics\json_comp...


In [1]:
import requests
import json

In [4]:
# Configuration
BASE_URL = "http://85.214.60.83:8000"
TOOLS_CALL_ENDPOINT = f"{BASE_URL}/tools/call"
TOOLS_LIST_ENDPOINT = f"{BASE_URL}/tools/list"
response = requests.get(TOOLS_LIST_ENDPOINT)
response.status_code

200

In [5]:
print("Status Code:", response.status_code)
print("\nAvailable Tools:")
print(json.dumps(response.json(), indent=2))

Status Code: 200

Available Tools:
{
  "tools": [
    {
      "name": "get_pokemon_info",
      "description": "Get detailed info about a Pok\u00e9mon by name"
    },
    {
      "name": "create_tournament_squad",
      "description": "Create a powerful squad of Pok\u00e9mon for a tournament"
    },
    {
      "name": "list_popular_pokemon",
      "description": "List popular tournament-ready Pok\u00e9mon"
    }
  ]
}


In [9]:
dir(response)


['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [10]:
response.content

b'Not Found'